In [1]:
import sys
import pandas as pd
import cv2 as cv
import numpy as np
from tqdm.notebook import tqdm
from skimage.feature import local_binary_pattern

sys.path.append(os.path.join(os.path.dirname(os.getcwd()),"modules"))

In [2]:
import NbrRegionSegment as seg
from MarinePredator import MPA

In [5]:
#fitness function to be used in the marine predator algo
def fitness_func(wr, wg, wc, thres):
    #read the csv file
    data = pd.read_csv("../CSV/reduced.csv")

    #fitness for each image against each predator
    fit = list()

    #fetch the image file
    for entry in tqdm(data.values):
        #read the image
        img = cv.imread("../Dataset/" + entry[2] + ".jpg")

        gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        #LBP part
        radius = 1
        points = 8*radius 

        lbp = local_binary_pattern(gray_img, points, radius, "uniform").astype("uint8")

        #segment image
        r = seg.RegionExtract(img, lbp, thres, wr, wg, wc)
        r1, r2 = r

        #find the fitness level
        intra, inter = seg.GetAllSegVariance(gray_img, r1, r2.shape[0], True)

        #push the fitness into the list
        fit.append(intra + inter) #-> fitness is the sum of the inter and intra variance and is a decreasing function
    
    #return the fitness as the avg of all fitness
    print(np.mean(fit))
    return np.mean(fit)

In [6]:
fitness_func(0.4, 0.3, 0.8, 25)

  0%|          | 0/11 [00:00<?, ?it/s]

Time to Calculate =  0.05485820770263672
Time to Calculate =  0.05485200881958008
Time to Calculate =  0.1066889762878418
Time to Calculate =  0.0747981071472168
Time to Calculate =  0.19479107856750488
Time to Calculate =  1.027498483657837
Time to Calculate =  0.14058566093444824
Time to Calculate =  2.3184385299682617
Time to Calculate =  0.7120323181152344
Time to Calculate =  0.2572784423828125
Time to Calculate =  0.3500697612762451
2.5465697158466685


2.5465697158466685

In [4]:
#algo params for the MPA
max_iter = 30
num_agents = 15

#wr, wg, wc, thres
lb = [0, 0, 0, 10]
ub = [1, 1, 1, 50]
num_var = 4

#create a class object for the metaheuristic algorithm
mpa = MPA.MPA(num_agents, max_iter, lb, ub, num_var, fitness_func)

In [7]:
mpa.Prey.shape

(15, 4)

In [6]:
for i in tqdm(mpa.iter_gen()):
    pass

|          | 0/? [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

nan


  0%|          | 0/11 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [36]:
tuple(map(float,np.split(np.array(lb), 4, 0)))

(0.0, 0.0, 0.0, 10.0)